In [1]:
from utils import *

In [2]:
# load dataset
from datasets import load_dataset

dataset = load_dataset("tianyang/repobench_python_v1.1")

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# load model
model_name = "bigcode/starcoderbase-3b"

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto",        
    trust_remote_code=True,
    torch_dtype=torch.bfloat16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/data/tianyang/anaconda3/envs/repobench/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
prompt = construct_prompt(
    dataset['cross_file_first'][31],
    version="normal",
    tokenizer=tokenizer,
    max_prompt_length=7800,
)

In [5]:
print(prompt)

# Repo Name: see2023/Bert-VITS2-ext
# Path: config.py
class Resample_config:
class Preprocess_text_config:
class Bert_gen_config:
class Emo_gen_config:
class Train_ms_config:
class Webui_config:
class Server_config:
class Translate_config:
class Config:
    def __init__(self, in_dir: str, out_dir: str, sampling_rate: int = 44100):
    def from_dict(cls, dataset_path: str, data: Dict[str, any]):
    def __init__(
        self,
        transcription_path: str,
        cleaned_path: str,
        train_path: str,
        val_path: str,
        config_path: str,
        val_per_lang: int = 5,
        max_val_total: int = 10000,
        clean: bool = True,
    ):
    def from_dict(cls, dataset_path: str, data: Dict[str, any]):
    def __init__(
        self,
        config_path: str,
        num_processes: int = 2,
        device: str = "cuda",
        use_multi_device: bool = False,
    ):
    def from_dict(cls, dataset_path: str, data: Dict[str, any]):
    def __init__(
        self,
     

In [6]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

model_inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
model_outputs = model.generate(**model_inputs, max_new_tokens=128, do_sample=True, temperature=0.2, top_p=0.95)

generated_code = tokenizer.decode(model_outputs[0][model_inputs["input_ids"].shape[-1]:], skip_special_tokens=True)

print(generated_code)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


# Path: for_deploy/infer_utils.py
import sys
import torch
from transformers import (
    AutoModelForMaskedLM,
    AutoTokenizer,
    DebertaV2Model,
    DebertaV2Tokenizer,
    ClapModel,
    ClapProcessor,
)
from config import config
from text.japanese import text2sep_kata

class BertFeature:
    def __init__(self, model_path, language="ZH"):
        self.model_path = model_path
        self.language = language
        self.tokenizer = None
        self.model


In [7]:
model_inputs = tokenizer(f"<fim_prefix>{prompt}<fim_suffix><fim_middle>", return_tensors="pt").to(model.device)
model_outputs = model.generate(**model_inputs, max_new_tokens=128, do_sample=True, temperature=0.2, top_p=0.95)

generated_code = tokenizer.decode(model_outputs[0][model_inputs["input_ids"].shape[-1]:], skip_special_tokens=True)

print(generated_code)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


            sep, ph, accent = text2sep_kata(text)
        else:
            sep, ph, accent = text.split(), text.split(), None
        sep = sep[:config.bert_gen_config.max_sep_len]
        sep = sep + ["[SEP]"]
        sep = sep[:config.bert_gen_config.max_sep_len]
        sep = sep + ["[CLS]"]
        sep = sep[:config.bert_gen_config.max_sep_len]
        sep = sep + ["[SEP]"]
        sep = sep[:config
